# EPII Protocol Example with Chronicle Dashboard

This notebook demonstrates how to use the EPII (Experiment Platform Intelligence Interface) protocol to execute quantum experiments remotely and monitor them in real-time using the Chronicle dashboard.

## Prerequisites

1. Start the EPII daemon with Chronicle integration:
```bash
python scripts/epii_chronicle_daemon.py --launch-viewer
```

2. Open the Chronicle dashboard at http://localhost:8051

3. Run this notebook to execute experiments

## Setup and Imports

In [ ]:
import grpc
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, HTML

# Import EPII protocol
import sys
sys.path.append('..')
from leeq.epii.proto import epii_pb2, epii_pb2_grpc

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## Connect to EPII Daemon

In [ ]:
# Connect to EPII service
channel = grpc.insecure_channel('localhost:50051')
stub = epii_pb2_grpc.ExperimentPlatformServiceStub(channel)

# Test connection
try:
    response = stub.Ping(epii_pb2.Empty())
    print("✓ Connected to EPII service")
    print(f"  Service: {response.message}")
    print(f"  Time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(response.timestamp/1000))}")
except grpc.RpcError as e:
    print(f"✗ Failed to connect: {e.code()} - {e.details()}")
    print("\nPlease start the daemon first:")
    print("  python scripts/epii_chronicle_daemon.py --launch-viewer")

## Query Service Capabilities

In [ ]:
# Get service capabilities
response = stub.GetCapabilities(epii_pb2.Empty())

print(f"Framework: {response.framework_name} v{response.framework_version}")
print(f"EPII Version: {response.epii_version}")
print(f"Backends: {', '.join(response.supported_backends)}")
print(f"\nAvailable Experiments ({len(response.experiment_types)}):")

for exp in response.experiment_types[:10]:  # Show first 10
    params = [p.name for p in exp.parameters]
    print(f"  • {exp.name}: {', '.join(params[:3])}...")

## Helper Functions

In [ ]:
def run_experiment(experiment_type, parameters, return_data=True):
    """Helper function to run an experiment via EPII."""
    request = epii_pb2.ExperimentRequest(
        experiment_type=experiment_type,
        parameters={k: str(v) for k, v in parameters.items()},
        return_raw_data=return_data
    )
    
    start_time = time.time()
    response = stub.RunExperiment(request)
    duration = time.time() - start_time
    
    if response.success:
        print(f"✓ {experiment_type} completed in {duration:.2f}s")
        return response
    else:
        print(f"✗ {experiment_type} failed: {response.error_message}")
        return None

def extract_data(response):
    """Extract measurement data from response."""
    if response and response.measurement_data:
        data = response.measurement_data[0]
        array = np.frombuffer(data.data, dtype=np.float64)
        return array.reshape(data.shape)
    return None

def display_results(response, title="Experiment Results"):
    """Display calibration results from experiment."""
    if response and response.calibration_results:
        print(f"\n{title}:")
        for key, value in response.calibration_results.items():
            print(f"  • {key}: {value:.4f}")

## Experiment 1: Resonator Spectroscopy

Find the resonator frequency for readout

In [ ]:
# Run resonator spectroscopy
response = run_experiment(
    "ResonatorSweepTransmission",
    {
        "qubit": "q0",
        "start": 9640,  # MHz
        "stop": 9655,
        "step": 0.5,
        "num_avs": 1000,
        "amp": 0.03
    }
)

display_results(response, "Resonator Spectroscopy")

# Plot if data available
data = extract_data(response)
if data is not None and len(data) > 0:
    frequencies = np.linspace(9640, 9655, len(data))
    plt.figure(figsize=(10, 4))
    plt.plot(frequencies, data)
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('Transmission')
    plt.title('Resonator Spectroscopy')
    plt.grid(True, alpha=0.3)
    plt.show()

print("\n📊 Check Chronicle dashboard for live updates!")

## Experiment 2: Rabi Oscillation

Calibrate the qubit π pulse amplitude

In [ ]:
# Run Rabi oscillation
response = run_experiment(
    "NormalisedRabi",
    {
        "qubit": "q0",
        "start": 0,
        "stop": 0.5,
        "step": 0.01,
        "num_avs": 500
    }
)

display_results(response, "Rabi Calibration")

# Plot Rabi oscillation
data = extract_data(response)
if data is not None and len(data) > 0:
    amplitudes = np.linspace(0, 0.5, len(data))
    plt.figure(figsize=(10, 4))
    plt.plot(amplitudes, data, 'o-', markersize=3)
    plt.xlabel('Pulse Amplitude')
    plt.ylabel('Population')
    plt.title('Rabi Oscillation')
    plt.grid(True, alpha=0.3)
    plt.show()

print("\n📊 Check Chronicle dashboard for live updates!")

## Experiment 3: Ramsey - Frequency Calibration

Fine-tune the qubit frequency

In [ ]:
# Run Ramsey experiment
response = run_experiment(
    "SimpleRamsey",
    {
        "qubit": "q0",
        "stop": 3,  # microseconds
        "step": 0.05,
        "set_offset": 1,  # MHz
        "num_avs": 500
    }
)

display_results(response, "Ramsey Frequency Calibration")

# Plot Ramsey fringes
data = extract_data(response)
if data is not None and len(data) > 0:
    times = np.linspace(0, 3, len(data))
    plt.figure(figsize=(10, 4))
    plt.plot(times, data, 'o-', markersize=3)
    plt.xlabel('Evolution Time (μs)')
    plt.ylabel('Population')
    plt.title('Ramsey Fringes')
    plt.grid(True, alpha=0.3)
    plt.show()

print("\n📊 Check Chronicle dashboard for live updates!")

## Experiment 4: T1 Relaxation Time

In [ ]:
# Measure T1
response = run_experiment(
    "SimpleT1",
    {
        "qubit": "q0",
        "time_length": 100,  # microseconds
        "time_resolution": 2,
        "num_avs": 500
    }
)

display_results(response, "T1 Measurement")

# Plot T1 decay
data = extract_data(response)
if data is not None and len(data) > 0:
    times = np.linspace(0, 100, len(data))
    plt.figure(figsize=(10, 4))
    plt.plot(times, data, 'o-', markersize=3)
    plt.xlabel('Delay Time (μs)')
    plt.ylabel('Population')
    plt.title('T1 Relaxation')
    plt.grid(True, alpha=0.3)
    
    # Add exponential fit line if T1 is available
    if response.calibration_results and 't1' in response.calibration_results:
        t1 = response.calibration_results['t1']
        fit = np.exp(-times / t1)
        plt.plot(times, fit, 'r--', alpha=0.7, label=f'T1 = {t1:.1f} μs')
        plt.legend()
    
    plt.show()

print("\n📊 Check Chronicle dashboard for live updates!")

## Experiment 5: Spin Echo (T2)

In [ ]:
# Measure T2 with spin echo
response = run_experiment(
    "SpinEcho",
    {
        "qubit": "q0",
        "free_evolution_time": 50,  # microseconds
        "time_resolution": 1,
        "num_avs": 500
    }
)

display_results(response, "Spin Echo (T2) Measurement")

# Plot echo decay
data = extract_data(response)
if data is not None and len(data) > 0:
    times = np.linspace(0, 50, len(data))
    plt.figure(figsize=(10, 4))
    plt.plot(times, data, 'o-', markersize=3)
    plt.xlabel('Evolution Time (μs)')
    plt.ylabel('Population')
    plt.title('Spin Echo Decay')
    plt.grid(True, alpha=0.3)
    
    # Add exponential fit line if T2 is available
    if response.calibration_results and 't2_echo' in response.calibration_results:
        t2 = response.calibration_results['t2_echo']
        fit = np.exp(-times / t2)
        plt.plot(times, fit, 'r--', alpha=0.7, label=f'T2 = {t2:.1f} μs')
        plt.legend()
    
    plt.show()

print("\n📊 Check Chronicle dashboard for live updates!")

## Interactive Parameter Exploration

Use widgets to interactively explore experiment parameters

In [ ]:
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

def interactive_rabi(amplitude_max=0.5, num_points=51, num_avs=500):
    """Interactive Rabi experiment."""
    
    # Run experiment
    response = run_experiment(
        "NormalisedRabi",
        {
            "qubit": "q0",
            "start": 0,
            "stop": amplitude_max,
            "step": amplitude_max / num_points,
            "num_avs": num_avs
        }
    )
    
    if response:
        data = extract_data(response)
        if data is not None:
            amplitudes = np.linspace(0, amplitude_max, len(data))
            plt.figure(figsize=(8, 4))
            plt.plot(amplitudes, data, 'o-', markersize=4)
            plt.xlabel('Pulse Amplitude')
            plt.ylabel('Population')
            plt.title(f'Rabi Oscillation (max_amp={amplitude_max:.2f})')
            plt.grid(True, alpha=0.3)
            plt.ylim([0, 1])
            plt.show()
            
            if response.calibration_results and 'pi_amp' in response.calibration_results:
                print(f"\nπ amplitude: {response.calibration_results['pi_amp']:.4f}")

# Create interactive widget
interact(
    interactive_rabi,
    amplitude_max=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description='Max Amp:'),
    num_points=IntSlider(min=11, max=101, step=10, value=51, description='Points:'),
    num_avs=IntSlider(min=100, max=2000, step=100, value=500, description='Averages:')
);

## Summary and Chronicle Dashboard

All experiments executed in this notebook are automatically recorded in the Chronicle session and can be viewed in the dashboard.

In [ ]:
# Display dashboard link
display(HTML("""
<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h3>📊 Chronicle Dashboard</h3>
    <p>View your experiments in real-time at:</p>
    <a href="http://localhost:8051" target="_blank" style="
        display: inline-block;
        padding: 10px 20px;
        background-color: #007bff;
        color: white;
        text-decoration: none;
        border-radius: 5px;
        font-weight: bold;
    ">Open Chronicle Dashboard</a>
    
    <h4 style="margin-top: 20px;">Features:</h4>
    <ul>
        <li>Live experiment monitoring (5-second refresh)</li>
        <li>Hierarchical experiment tree view</li>
        <li>Interactive plot visualization</li>
        <li>Experiment details and parameters</li>
    </ul>
</div>
"""))

print("\n✅ Notebook execution complete!")
print("All experiments have been sent via EPII and recorded in Chronicle.")

## Cleanup

In [ ]:
# Close the gRPC channel
channel.close()
print("✓ gRPC channel closed")
print("\nThe EPII daemon and Chronicle dashboard remain running.")
print("To stop them, press Ctrl+C in the daemon terminal.")